# 準備工作

In [1]:
# 準備工作

from bs4 import BeautifulSoup
import time
import datetime
import re
import sqlite3
import os
import json
import csv
import pprint

data_source = "../data/pacific.json"

In [2]:
# 讀取全臺縣市區域街道查詢表

addr_source = "../data/全台鄉鎮市區街道列表.csv"

with open(addr_source) as addr_file:    
    addr_ref = csv.reader(addr_file)
    addr_ref = [tuple(row) for row in addr_ref]

# 定義方法

In [3]:
def sect_format(string):
    final_string = string
    before_list = ['1段','2段','3段','4段','5段','6段','7段','8段','9段','台']
    after_list = ['一段','二段','三段','四段','五段','六段','七段','八段','九段','臺']
    
    bef_len = len(before_list)
    aft_len = len(after_list)
    
    if bef_len == aft_len:
        for i in range(0,bef_len):
            final_string = final_string.replace(before_list[i],after_list[i])
    else:
        print("before_list 長度與 after_list 長度不同, 請檢查")
            
    return final_string

In [4]:
def which_in_list(addr,addr_ref):

    final_addr = ""
    for addr_tuple in addr_ref:
        
        city_distr = addr_tuple[0] + addr_tuple[1]
        
        if addr_tuple[1] + addr_tuple[2] in addr:
            final_addr = "".join(addr_tuple)
            break
        elif city_distr in addr:
            if len(city_distr) > len(final_addr):
                final_addr = addr_tuple[0] + addr_tuple[1]
                
    return final_addr

In [5]:
def findall_and_glue(regex,string):
    final_string = ""
    try:
        final_list = re.findall(regex,test_string)
        for each in final_list:
            final_string += each
        return final_string
    except:
        return ""

In [6]:
def clean_address(addr):

    addr = sect_format(addr)
    final_addr = ""
    final_addr += which_in_list(addr,addr_ref)
    final_addr += findall_and_glue('\d+巷|\d+弄|\d+鄰|\d+號|\d+樓',addr)

    return final_addr

In [7]:
# 定義 permits() 方法
# 判斷住所是否與許item_string中指定的物件(如寵物等)
# main_data_all為包含單一住所全部資料的dictionary

def permits(main_data_all, item_string):

    permit = 'NULL'
    re_no_criteria = '(禁止\w*?' + item_string + '|不可\w*?' + item_string + ')'
    re_yes_criteria = '(開放\w*?' + item_string + '|可\w*?' + item_string + ')'
    
    for each in main_data_all:
        desc_value = str("".join(main_data_all[each]))
        if item_string in each or item_string in desc_value:
            if re.search(re_no_criteria, each + " " + desc_value):
                permit = "N"
            elif re.search(re_yes_criteria, each + " " + desc_value):
                permit = "Y"
    
    return permit

In [8]:
# 定義 has() 方法
# 判斷住所是否含有item_string中指定的物件(如停車場等)
# main_data_all為包含單一住所全部資料的dictionary

def has(main_data_all,item_string):
    
    permit = 'NULL'
    re_no_criteria = '(沒有\w*?' + item_string + '|無\w*?' + item_string + ')'
    re_yes_criteria = '(有\w*?' + item_string + '|含\w*?' + item_string + ')'
    
    for each in main_data_all:
        desc_value = str("".join(main_data_all[each]))
        if item_string in each or item_string in desc_value:
            if re.findall(re_no_criteria, each + " " + desc_value):
                permit = "N"
            elif re.findall(re_yes_criteria, each + " " + desc_value):
                permit = "Y"
                
    return permit

In [9]:
# get_filtered_dict() 方法
# 從main_data資料集中取出所有MySQL所需的資料
# 並以dictionary格式傳回

def get_filtered_dict(main_data):
    
    filtered_dict = {}

    for each_id in main_data:

        main_data_all = main_data[each_id]
        rent_desc = main_data_all['出租說明']
        publ_facil_desc = main_data_all['公共設施']
        priv_facil_desc = main_data_all['居家設備']
        furniture_desc = main_data_all['傢俱']
        
        
        # url

        url = 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=' + each_id

        
        # title
        
        title = main_data_all['標題']

        
        # address
        
        address = clean_address(main_data_all['地址'])
        
        
        # 格局
        
        try:
            格局 = main_data_all['格局']
        except:
            格局 = 'NULL'
        
        
        # ttype       
        
        try:
            ttype = main_data_all['出租型態']            
            if ttype in ['雅房', '獨立套房', '整樓(棟)住家']:
                ttype = "H"
            elif ttype in ['店面商辦']:
                ttype = "S"
            elif ttype in ['工廠','出租土地']:
                ttype = "F"
            else:
                # 其他判斷方式
                if re.search('商業|店面|店',title):
                    ttype = "S"                
                elif re.search('工業|工業地|廠房|田地|農舍|農地',title):
                    ttype = "F"
                elif re.search('辦公|大樓',title):
                    ttype = "O"
                else:
                    ttype = "H"           
                
        except:
            print(each_id + " 沒有種類相關資料")
            ttype = "NULL"


        # avg_rent

        avg_rent = main_data_all['價格']
        avg_rent = re.search('(\d|\,)+',avg_rent)
        avg_rent = avg_rent.group(0)
        avg_rent = avg_rent.replace(",","")
        avg_rent = int(avg_rent)
        

        # sex
        
        try:
            if '限男性' in rent_desc:
                sex = "M"
            elif '限女性' in rent_desc:
                sex = "F"
            elif '男女不拘' in rent_desc:
                sex = "B"
            else:
                # 如果在'出租說明'找不到 就到'房屋特色'找
                detailed_desc = main_data_all['房屋特色']
                
                if re.search('限男性', detailed_desc):
                    sex = "M"
                if re.search('限女性', detailed_desc):
                    sex = "F"
                elif re.search('男女不拘|男女不限|不限男女', detailed_desc):
                    sex = "B"                
                else:
                    sex = "NULL"
        except:
            sex = "NULL"


        # space    

        try:
            space = main_data_all['坪數']
            space = re.search('^(\d|\,|\.)*',space)
            space = space.group(0)
        except:
            space = "NULL"


        # other info        
        
        # smoke

        smoke = permits(main_data_all,'抽煙')
        smoke = permits(main_data_all,'吸煙')
        
        
        # pet
        
        if '可養寵物' in rent_desc:
            pet = 'Y'
        else:         
            pet = permits(main_data_all,'寵物')
        
        
        # cook
        
        cook = permits(main_data_all,'開伙')
        
        
        # parking
        
        parking = has(main_data_all,'車位')

    
        # insert values

        filtered_dict.update({each_id:{
            'url':url,
            'title':title,
            'address':address,
            '格局':格局,            
            'ttype':ttype,
            'avg_rent':avg_rent,
            'sex':sex,
            'space':space,
            'smoke':smoke,
            'pet':pet,
            'cook':cook,
            'parking':parking        
        }})
    
    return filtered_dict

# 執行程式

In [10]:
# 讀取已存好的資料

with open(data_source) as data_file:    
    main_data = json.load(data_file)

In [11]:
# 檢視get_filtered_dict()結果

pprint.pprint(get_filtered_dict(main_data))

{'060496': {'address': '桃園市桃園區鎮四街',
            'avg_rent': 1150,
            'cook': 'NULL',
            'parking': 'N',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '43.87',
            'title': '東區全新優質大3房(二)',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=060496',
            '格局': '3房 2廳 2衛'},
 '060499': {'address': '臺北市中山區長春路',
            'avg_rent': 650000,
            'cook': 'NULL',
            'parking': 'N',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '387.19',
            'title': '長春商業店面',
            'ttype': 'S',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=060499',
            '格局': '  1衛'},
 '060512': {'address': '桃園市中壢區聖溪二路',
            'avg_rent': 496,
            'cook': 'NULL',
            'parking': 'Y',
            'pet': 'NULL',
            'sex': 'NULL

            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '42.92',
            'title': '東方荷蘭',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=061439',
            '格局': '4房 2廳 2衛'},
 '061446': {'address': '南投縣南投市信義街',
            'avg_rent': 2580,
            'cook': 'NULL',
            'parking': 'N',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '191.69',
            'title': '信義街角間金店面',
            'ttype': 'S',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=061446',
            '格局': '10房 3廳 2衛'},
 '061447': {'address': '彰化縣員林市育英路',
            'avg_rent': 298,
            'cook': 'NULL',
            'parking': 'NULL',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '41.62',
            'title': '建國路旁丁建',
            'ttype': 'H',
            'url': 'https://www

            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=062013',
            '格局': '3房 2廳 2衛'},
 '062014': {'address': '南投縣南投市東山路',
            'avg_rent': 398,
            'cook': 'NULL',
            'parking': 'N',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '32.26',
            'title': '南投景觀華廈 附平面車位',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=062014',
            '格局': '3房 2廳 2衛'},
 '062039': {'address': '新北市新莊區中正路',
            'avg_rent': 4680,
            'cook': 'NULL',
            'parking': 'N',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '57.57',
            'title': 'C182中正路金店面',
            'ttype': 'S',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=062039',
            '格局': '5房 2廳 3衛'},
 '062057': {'address': '苗栗縣竹南鎮平等街',
            'a

            '格局': '  '},
 '062595': {'address': '嘉義縣水上鄉外溪洲',
            'avg_rent': 4305,
            'cook': 'NULL',
            'parking': 'NULL',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '633.13',
            'title': '中華路工建',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=062595',
            '格局': '  '},
 '062600': {'address': '桃園市桃園區藝文一街',
            'avg_rent': 4680,
            'cook': 'NULL',
            'parking': 'Y',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '109.74',
            'title': '昭揚乾坤商辦',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=062600',
            '格局': '  '},
 '062602': {'address': '新北市中和區中正路',
            'avg_rent': 1468,
            'cook': 'NULL',
            'parking': 'Y',
            'pet': 'NULL',
           

            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '56.78',
            'title': '狀元科學美墅',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=062955',
            '格局': '3房 2廳 2.5衛'},
 '062956': {'address': '宜蘭縣蘇澳鎮海山西路',
            'avg_rent': 938,
            'cook': 'NULL',
            'parking': 'Y',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '59.01',
            'title': '海山東路店住',
            'ttype': 'S',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=062956',
            '格局': '4房 1廳 5衛'},
 '062961': {'address': '宜蘭縣冬山鄉鹿安路',
            'avg_rent': 835,
            'cook': 'NULL',
            'parking': 'Y',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '38.26',
            'title': '清溝國小宏泰透天',
            'ttype': 'H',
     

...

            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=068582',
            '格局': '4房 2廳 2衛'},
 '068583': {'address': '彰化縣彰化市彰南路五段',
            'avg_rent': 1800,
            'cook': 'NULL',
            'parking': 'NULL',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '300.45',
            'title': '★快官渡假山莊',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=068583',
            '格局': '  '},
 '068584': {'address': '彰化縣彰化市民族路',
            'avg_rent': 3880,
            'cook': 'NULL',
            'parking': 'Y',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '75.9',
            'title': '正民族路地61坪金店',
            'ttype': 'S',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=068584',
            '格局': '6房 3廳 7衛'},
 '068585': {'address': '新北市林口區中華一路',
            'avg_r

            'cook': 'NULL',
            'parking': 'Y',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '59.4',
            'title': '桃園區漂亮景觀大三房+平車大樓',
            'ttype': 'O',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=068836',
            '格局': '3房 2廳 2衛'},
 '068837': {'address': '新北市新店區北新路一段',
            'avg_rent': 1828,
            'cook': 'NULL',
            'parking': 'N',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '26.63',
            'title': '中華庭院一樓',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=068837',
            '格局': '3房 2廳 2衛'},
 '068839': {'address': '桃園市楊梅區裕成南路',
            'avg_rent': 380,
            'cook': 'NULL',
            'parking': 'Y',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '35.41',

            'ttype': 'S',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=069056',
            '格局': '1房 1廳 1衛'},
 '069057': {'address': '新北市五股區成泰路三段',
            'avg_rent': 2480,
            'cook': 'NULL',
            'parking': 'N',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '25.7',
            'title': 'E193三角窗金店面~稀有釋出',
            'ttype': 'S',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=069057',
            '格局': ' 1廳 1衛'},
 '069059': {'address': '新北市新莊區中央路',
            'avg_rent': 3068,
            'cook': 'NULL',
            'parking': 'Y',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '62.02',
            'title': 'B72新莊副都心~新富邑15F',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=069059',
            '格局': '  '},
 '069060': {'address':

            'avg_rent': 2380,
            'cook': 'NULL',
            'parking': 'Y',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '89.11',
            'title': '9.9米大面寬獨棟別墅',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=069289',
            '格局': '8房 4廳 4衛'},
 '069291': {'address': '新竹縣竹東鎮自由街',
            'avg_rent': 198,
            'cook': 'NULL',
            'parking': 'N',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '12.64',
            'title': '元邦套房II',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=069291',
            '格局': '1房 1廳 1衛'},
 '069292': {'address': '臺南市永康區大仁街',
            'avg_rent': 3498,
            'cook': 'NULL',
            'parking': 'NULL',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
  

            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=071162',
            '格局': '  '},
 '071163': {'address': '新北市板橋區光華街',
            'avg_rent': 1350,
            'cook': 'NULL',
            'parking': 'Y',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '40.14',
            'title': 'E237林家花園邊間美裝潢4房2衛+車位',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=071163',
            '格局': '4房 2廳 2衛'},
 '071165': {'address': '新北市泰山區明志路二段',
            'avg_rent': 698,
            'cook': 'NULL',
            'parking': 'N',
            'pet': 'NULL',
            'sex': 'NULL',
            'smoke': 'NULL',
            'space': '28.16',
            'title': '靜巷學區旁美寓',
            'ttype': 'H',
            'url': 'https://www.pacific.com.tw/Object/ObjectRentDetail/?saleID=071165',
            '格局': '3房 2廳 2衛'},
 '071166': {'address': 